In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from dotenv import load_dotenv
import os
import shutil

load_dotenv()

True

In [4]:
pdf_document = "./docs/Resumen IC2.pdf"
persist_dir = "./chroma_db"

GITHUB_MODEL = "gpt-4o-mini"
GITHUB_EMBEDDINGS = "text-embedding-3-small"
GITHUB_API_KEY = os.getenv("GITHUB_TOKEN")
GITHUB_MODEL_URL = os.getenv("GITHUB_MODEL_URL")

In [5]:
loader = PyPDFLoader(pdf_document)
pages = loader.load()

# async for page in loader.alazy_load():
#     pages.append(page)
    
print(f"Cargadas {len(pages)} páginas del documento PDF.")

Cargadas 108 páginas del documento PDF.


In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chunks = splitter.split_documents(pages)
print(f"Dividido en {len(chunks)} fragmentos de texto.\n")

Dividido en 143 fragmentos de texto.



In [7]:
llm = ChatOpenAI(model=GITHUB_MODEL, api_key=GITHUB_API_KEY, temperature=0, base_url=GITHUB_MODEL_URL)
embeddings = OpenAIEmbeddings(model=GITHUB_EMBEDDINGS, api_key=GITHUB_API_KEY, base_url=GITHUB_MODEL_URL)

In [8]:
if os.path.exists(persist_dir):
    shutil.rmtree(persist_dir)
    Chroma(persist_directory=persist_dir, embedding_function=embeddings).delete_collection()
    
    print(f"Se eliminó la colección existente en {persist_dir}")

try:
    vectorstore = Chroma.from_documents(
        documents=chunks,  # Ahora usamos todos los chunks
        embedding=embeddings,
        persist_directory=persist_dir,
        collection_name="openai_collection"
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
except Exception as e:
    print(f"❌ Error: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

In [11]:
print(retriever.invoke("¿Qué es RISC?")[2].page_content)

Ejemplo: Para simplificar las operaciones que se re alizan en la etapa de extracción y en la decodificación, el 
PowerPC970 cuenta con una etapa de predecodificación situada delate de la I -caché y del buffer de prefetch. 
Esta etapa se ocupa de añadir 5 bits a cada instrucción, que se utilizan posteriormente en la etapa de Fetch para 
determinar el tipo de instrucciones de salto y proceder o no a su especulación. 
 
 
 
2.4.2 Traducción de instrucciones 
 
Es el proceso por el que una instrucción RISC o CISC se convierte en un grupo equivalente de instrucciones 
básicas RISC. Estas operaciones se conocen como: 
 
- Microoperaciones (micro-ops) en terminología Intel. 
- Operaciones internas (IOPs - Internal Operations) en terminología PowerPC 
- ROPs (RIPS operations) en terminología AMD. 
 
Esta técnica se utiliza para redu cir la complejidad de determinadas instrucciones. La idea que subyace es 
simplificar el repertorio original de instrucciones para que su procesamiento hardware se

In [ ]:
aware_retriever_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Genera una consulta de búsqueda para recuperar información relevante basada en la conversación."),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            Responde brevemente a la pregunta del usuario basándote en el contexto proporcionado. 
            Si no sabes la respuesta, di 'No lo sé' o 'No tengo información al respecto.'
            No inventes respuestas.
            Contexto: {context}
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

In [40]:
print("- Template del prompt del retriever:\n", aware_retriever_prompt.messages[0].prompt.template, "\n")
print("- Variables del prompt del retriever:\n", aware_retriever_prompt.input_variables, "\n")
print("-" * 50, "\n")
print("- Template del prompt de QA:\n", qa_prompt.messages[0].prompt.template, "\n")
print("- Variables del prompt de QA:\n", qa_prompt.input_variables, "\n")

- Template del prompt del retriever:
 Genera una consulta de búsqueda para recuperar información relevante basada en la conversación. 

- Variables del prompt del retriever:
 ['chat_history', 'input'] 

-------------------------------------------------- 

- Template del prompt de QA:
 
                Responde brevemente a la pregunta del usuario basándote en el contexto proporcionado. 
                Si no sabes la respuesta, di 'No lo sé' o 'No tengo información al respecto.'
                No inventes respuestas.
                Contexto: {context}
             

- Variables del prompt de QA:
 ['chat_history', 'context', 'input'] 



In [ ]:
# Ahora crear el history aware retriever
try:
    history_aware_retriever = create_history_aware_retriever(
        retriever=retriever,
        llm=llm,
        prompt=aware_retriever_prompt
    )
    print("✅ History-aware retriever creado exitosamente!")
except Exception as e:
    print(f"❌ Error: {e}")
    raise

Variables del aware_retriever_prompt: ['chat_history', 'input']
Variables del prompt corregido: ['chat_history', 'input']
✅ History-aware retriever creado exitosamente!


In [41]:
documents_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)
print(documents_chain)

bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.me

In [50]:
retrieval_chain = create_retrieval_chain(
    retriever=history_aware_retriever,
    combine_docs_chain=documents_chain
)
print(retrieval_chain)

bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x74853be71e80>, search_kwargs={'k': 3}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_c

In [54]:
chat_history = []

def ask_to_chatbot(question: str):
    inputs = {
        "input": question,
        "chat_history": chat_history
    }
    result = retrieval_chain.invoke(inputs)
    answer = result["answer"]
    chat_history.append(HumanMessage(content=question))
    chat_history.append(AIMessage(content=answer))
    return answer

In [56]:
print(ask_to_chatbot("¿Qué es RISC?"))
print(ask_to_chatbot("¿Y CISC?"))
print(ask_to_chatbot("¿Qué te pregunté antes?"))
print(ask_to_chatbot("¿Y después de preguntarte por RISC, por qué te pregunté?"))

RISC (Reduced Instruction Set Computer) es una arquitectura de procesador que se caracteriza por un conjunto de instrucciones sencillas y de formato fijo, lo que permite una ejecución más rápida y eficiente. Su diseño busca reducir la cantidad de accesos a memoria mediante un aumento en la cantidad de registros internos en el procesador y facilita la implementación de la ejecución segmentada de instrucciones (pipeline). Esto resulta en un hardware más simple y un mayor rendimiento en comparación con arquitecturas más complejas como CISC (Complex Instruction Set Computer).
CISC (Complex Instruction Set Computer) es una arquitectura de procesador que se caracteriza por tener un conjunto de instrucciones más complejo y potente, capaz de realizar múltiples operaciones en una sola instrucción. Esta arquitectura fue diseñada para reducir el tamaño de los programas y la cantidad de instrucciones que debían almacenarse, lo que era importante en los primeros computadores con memoria limitada. S